# Regression: Multiple Regression Part 2

In [1]:
# Fire up Turi Create
import turicreate

In [2]:
# Load in house sales data
sales = turicreate.SFrame('kc_house_data.csv')

Finished parsing file /home/ppratikbp/Coursera Machine Learning/C2_2_Multiple Regression/kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.221315 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/ppratikbp/Coursera Machine Learning/C2_2_Multiple Regression/kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.19428 secs.

In [3]:
# Convert to Numpy Array
import numpy as np # note this allows us to refer to numpy as np instead 

Now we will write a function that will accept an SFrame, a list of feature names (e.g. ['sqft_living', 'bedrooms']) and an target feature e.g. ('price') and will return two things:
* A numpy matrix whose columns are the desired features plus a constant column (this is how we create an 'intercept')
* A numpy array containing the values of the output

With this in mind, complete the following function (where there's an empty line you should write a line of code that does what the comment above indicates)

In [7]:
# Converts SFrame into numpy array and select features
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

In [8]:
# print out value of function
get_numpy_data(sales, ['sqft_living'], 'price')

(array([[   1, 1180],
        [   1, 2570],
        [   1,  770],
        ...,
        [   1, 1020],
        [   1, 1600],
        [   1, 1020]]),
 array([221900., 538000., 180000., ..., 402101., 400000., 325000.]))

In [10]:
# For testing let's use the 'sqft_living' feature and a constant as our features and price as our output:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') # the [] around 'sqft_living' makes it a list
print(example_features[0,:]) # this accesses the first row of the data the ':' indicates 'all columns'
print(example_output[0]) # and the corresponding output

[   1 1180]
221900.0


In [11]:
# Predicting output given regression weights
my_weights = np.array([1., 1.]) # the example weights
my_features = example_features[0,] # we'll use the first data point
predicted_value = np.dot(my_features, my_weights)
print(predicted_value)

1181.0


. Recall that the predictions from all the observations is just the RIGHT (as in weights on the right) dot product between the features *matrix* and the weights *vector*. With this in mind finish the following predict_output function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [13]:
# np.dot() also works when dealing with a matrix and a vector
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

In [14]:
# If you want to test your code run the following cell:
test_predictions = predict_output(example_features, my_weights)
print(test_predictions[0]) # should be 1181.0
print(test_predictions[1]) # should be 2571.0

1181.0
2571.0


In [16]:
# Computing the Derivative
def feature_derivative(errors, feature):
    # Assume that errors and feature are both numpy arrays of the same length (number of data points)
    # compute twice the dot product of these vectors as 'derivative' and return the value
    derivative = 2*(np.dot(errors, feature)) # From the lecture
    return(derivative)

In [17]:
# To test your feature derivartive run the following:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') 
my_weights = np.array([0., 0.]) # this makes all the predictions 0
test_predictions = predict_output(example_features, my_weights) 
# just like SFrames 2 numpy arrays can be elementwise subtracted with '-': 
errors = test_predictions - example_output # prediction errors in this case is just the -example_output
feature = example_features[:,0] # let's compute the derivative with respect to 'constant', the ":" indicates "all rows"
derivative = feature_derivative(errors, feature)
print(derivative)
print(-np.sum(example_output)*2) # should be the same as derivative

-23345850016.0
-23345850016.0


Now we will write a function that performs a gradient descent. The basic premise is simple. Given a starting point we update the current weights by moving in the negative gradient direction.  

The amount by which we move in the negative gradient *direction*  is called the 'step size'. We stop when we are 'sufficiently close' to the optimum. We define this by requiring that the magnitude (length) of the gradient vector to be smaller than a fixed 'tolerance'.

With this in mind, complete the following gradient descent function below using your derivative function above. For each step in the gradient descent we update the weight for each feature befofe computing our stopping criteria

In [18]:
# Gradient Descent
# Recall that the gradient is the direction of *increase* and therefore the negative gradient is the direction 
# of *decrease* and  we're trying to *minimize* a cost function.
from math import sqrt # recall that the magnitude/length of a vector [g[0], g[1], g[2]] is sqrt(g[0]^2 + g[1]^2 + g[2]^2)

In [19]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    while not converged:
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        predictions = predict_output(feature_matrix, weights)
        # compute the errors as predictions - output
        errors = predictions - output
        gradient_sum_squares = 0 # initialize the gradient sum of squares
        # while we haven't reached the tolerance yet, update each feature's weight
        for i in range(len(weights)): # loop over each weight
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = 2*(np.dot(errors, feature_matrix[:, i]))
            # add the squared value of the derivative to the gradient sum of squares (for assessing convergence)
            gradient_sum_squares = gradient_sum_squares + (derivative)**2
            # subtract the step size times the derivative from the current weight
            weights = weights - step_size*derivative
        # compute the square-root of the gradient sum of squares to get the gradient magnitude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

In [20]:
# Running the Gradient Descent as Simple Regression
# First let's split the data into training and test data.
train_data,test_data = sales.random_split(.8,seed=0)

Although the gradient descent is designed for multiple regression since the constant is now a feature we can use the gradient descent function to estimate the parameters in the simple regression on squarefeet. 

In [21]:
# This cell sets up the feature_matrix, output, initial weights and step size for the first model:
# let's test out the gradient descent
simple_features = ['sqft_living']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [23]:
# Next run your gradient descent with the above parameters.
regression_gradient_descent(simple_feature_matrix, output, initial_weights,step_size,tolerance)

array([-46719.20069412,    281.79930588])

In [25]:
# Use your newly estimated weights and your predict_output() function to compute the predictions on all the TEST data 
# (you will need to create a numpy array of the test feature_matrix and test output first
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)

In [26]:
# Now compute your predictions using test_simple_feature_matrix and your weights from above.
mode_weights = regression_gradient_descent(test_simple_feature_matrix, test_output, initial_weights,step_size,tolerance)

In [27]:
# print model
mode_weights

array([-46718.75400369,    282.24599631])

In [29]:
# print predicted outputs
predict_output(test_simple_feature_matrix, mode_weights)[0:4]

array([356893.02072011, 785906.93511185, 435921.89968701, 608091.95743633])

In [30]:
# Now that you have the predictions on test data, compute the RSS on the test data set. Save this value for comparison later. Recall that RSS 
# is the sum of the squared errors (difference between prediction and output).
predictions_test = predict_output(test_simple_feature_matrix, mode_weights)
errors = predictions_test - test_output
RSS_prediction_test = sum(errors**2)
print(RSS_prediction_test)

275388789364303.25


In [31]:
# Running a multiple regression
## Now we will use more than one actual feature. Use the following code to
## produce the weights for a second model with the following parameters:
model_features = ['sqft_living', 'sqft_living15']
my_output = 'price'
(feature_matrix_mr, output_mr) = get_numpy_data(train_data, model_features,my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [32]:
# Use the above parameters to estimate the model weights. Record these values for your quiz.
predictions = predict_output(feature_matrix_mr, initial_weights)
errors = predictions - output

In [37]:
mreg = turicreate.linear_regression.create(train_data, 'price',   model_features, validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 2

Number of unpacked features : 2

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 1.008436     | 4588581.146247     | 261304.240584                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [38]:
# Use your newly estimated weights and the predict_output function to compute the predictions on the TEST data.
mreg.coefficients

name,index,value,stderr
(intercept),None,-100261.37811495864,6070.419353169284
sqft_living,None,245.18716945513242,3.2871359552854176
sqft_living15,None,65.27280201358893,4.412919722494058


In [40]:
# write get_residual_sum_of_squares function
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predicted_outcome = model.predict(data)
    # Then compute the residuals/errors
    residuals = data[outcome] - predicted_outcome
    # Then square and add them up
    RSS = sum((residuals)**2)
    return(RSS)   

In [42]:
# Get predicted value of the first prediction
mreg.predict(test_data)[0]

366541.86179006903

In [43]:
# What is the actual price for the 1st house in the test data set?
test_data['price'][0]

310000.0

In [44]:
# Now use your predictions and the output to compute the RSS for model 2 on TEST data.
get_residual_sum_of_squares(mreg, test_data, 'price')

270270381112021.72